# mounting to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# install library prerequisites

In [ ]:
!pip install numpy scikit-learn pillow tqdm

# imports

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image, UnidentifiedImageError
import pickle

# script for finding the folders in drive

In [ ]:
# Print the contents of the ASL_to_Text_Project directory
project_dir = '/content/drive/MyDrive/ASL_to_Text_Project'
print(f"\nContents of {project_dir}:")
print(os.listdir(project_dir))

# Print the contents of the data directory
data_dir = os.path.join(project_dir, 'data')
print(f"\nContents of {data_dir}:")
print(os.listdir(data_dir))


Contents of /content/drive/MyDrive/ASL_to_Text_Project:
['data', 'models']

Contents of /content/drive/MyDrive/ASL_to_Text_Project/data:
['labels', 'images']


# utilizing alexnet code

In [2]:
# --------------------- Configuration ---------------------
IMG_SIZE = 227  # AlexNet input size
BATCH_SIZE = 32
EPOCHS = 50
DATA_DIR = r"/content/drive/My Drive/ASL_to_Text_Project/data" # Changed to match the correct path in Google Drive
IMAGES_DIR = os.path.join(DATA_DIR, 'images')  # Path to the 'images' folder
MODEL_DIR = r"/content/drive/My Drive/ASL_to_Text_Project/models" # Changed to match the correct path in Google Drive
LABELS_DIR = r"/content/drive/My Drive/ASL_to_Text_Project/data/labels" # Changed to match the correct path in Google Drive
# --------------------- Configuration ---------------------

# alexnet model (tweakable to increase accuracy)
def create_alexnet_model(num_classes, input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    """Creates an AlexNet-based model for image classification."""

    input_layer = tf.keras.Input(shape=input_shape)

    # Convolutional Layer 1
    x = tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4,
                                activation='relu', padding='same')(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2)(x)

    # Convolutional Layer 2
    x = tf.keras.layers.Conv2D(filters=256, kernel_size=5, strides=1,
                                activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2)(x)

    # Convolutional Layers 3-5
    x = tf.keras.layers.Conv2D(filters=384, kernel_size=3, strides=1,
                                activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(filters=384, kernel_size=3, strides=1,
                                activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=1,
                                activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2)(x)

    # Flatten and Fully Connected Layers
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(4096, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(4096, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=input_layer, outputs=outputs)
    return model
# ADD REGULARIZATION TO AVOID OVERFIT
# good
def load_data(data_dir, allowed_extensions=('.jpg', '.jpeg', '.png')):
    """Loads images and their corresponding labels."""
    images = []
    labels = []
    for label in os.listdir(data_dir):
        if label.startswith('.'):
            continue
        label_dir = os.path.join(data_dir, label)
        if os.path.isdir(label_dir):
            for img_name in os.listdir(label_dir):
                if img_name.startswith('.'):
                    continue
                if not img_name.lower().endswith(allowed_extensions):
                    print(f"Skipping unsupported file: {img_name}")
                    continue
                try:
                    img_path = os.path.join(label_dir, img_name)
                    img = Image.open(img_path)
                    img.verify()
                    img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
                    img_array = img_to_array(img)
                    images.append(img_array)
                    labels.append(label)
                except (IOError, UnidentifiedImageError) as e:
                    print(f"Error loading image {img_name}: {e}")
                    continue
    return np.array(images), np.array(labels)


# --- Main Execution ---

# Load and preprocess data
X, y = load_data(IMAGES_DIR)
print(f"Number of images loaded: {len(X)}")
X = X / 255.0

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)
print(f"Number of Classes : {num_classes}")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

# Convert to categorical
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Create and compile model
model = create_alexnet_model(num_classes)
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode="nearest",
)

# Train the model
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(
                MODEL_DIR, "asl_model_{epoch:02d}_{val_accuracy:.2f}.keras"
            ),
            monitor="val_accuracy",
            save_best_only=True,
            mode="max",
        )
    ],
)

# Save the entire model
model.save(os.path.join(MODEL_DIR, "asl_model.h5"))

# Save the label encoder
with open(os.path.join(LABELS_DIR, 'label_encoder.pkl'), 'wb') as f:
  pickle.dump(le, f)

NameError: name 'os' is not defined

# past code with alex net

In [ ]:
# IMG_SIZE = 227
# BATCH_SIZE = 32
# EPOCHS = 100
# DATA_DIR = "/content/drive/MyDrive/ASL_to_Text_Project/data"
# MODEL_DIR = "/content/drive/MyDrive/ASL_to_Text_Project/models"
# LABELS_DIR = "/content/drive/MyDrive/ASL_to_Text_Project/data/labels"


# # Configuration

# def create_improved_model(num_classes):
#     model = Sequential([
#         Conv2D(64, 11, strides=4, padding="same", activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
#         BatchNormalization(),
#         MaxPooling2D(3, strides=2),
#         Conv2D(192, 5, padding="same", activation="relu"),
#         BatchNormalization(),
#         MaxPooling2D(3, strides=2),
#         Conv2D(384, 3, padding="same", activation="relu"),
#         BatchNormalization(),
#         Conv2D(256, 3, padding="same", activation="relu"),
#         BatchNormalization(),
#         Conv2D(256, 3, padding="same", activation="relu"),
#         BatchNormalization(),
#         MaxPooling2D(3, strides=2),
#         Flatten(),
#         Dense(4096, activation="relu"),
#         BatchNormalization(),
#         Dropout(0.5),
#         Dense(4096, activation="relu"),
#         BatchNormalization(),
#         Dropout(0.5),
#         Dense(num_classes, activation="softmax")
#     ])
#     return model

# def load_balanced_data(data_dir, img_size=IMG_SIZE, max_samples_per_class=1000):
#     images = []
#     labels = []
#     class_counts = {}
#     images_dir = os.path.join(data_dir, "images")

#     for label in os.listdir(images_dir):
#         label_dir = os.path.join(images_dir, label)
#         if os.path.isdir(label_dir):
#             class_counts[label] = 0
#             for img_name in os.listdir(label_dir):
#                 if class_counts[label] >= max_samples_per_class:
#                     break
#                 img_path = os.path.join(label_dir, img_name)
#                 if os.path.isfile(img_path):
#                     try:
#                         img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_size, img_size))
#                         img_array = tf.keras.preprocessing.image.img_to_array(img)
#                         images.append(img_array)
#                         labels.append(label)
#                         class_counts[label] += 1
#                     except Exception as e:
#                         print(f"Error loading image {img_path}: {e}")

#     print("Class distribution:")
#     for label, count in class_counts.items():
#         print(f"{label}: {count}")

#     return np.array(images), np.array(labels)

# def main():
#     X, y = load_balanced_data(DATA_DIR)

#     if len(X) == 0:
#         print("No images were loaded. Please check the data directory structure.")
#         return

#     print(f"Loaded {len(X)} images with shape {X.shape}")
#     print(f"Unique labels: {np.unique(y)}")

#     le = LabelEncoder()
#     y_encoded = le.fit_transform(y)
#     num_classes = len(le.classes_)

#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
#     )

#     X_train = X_train / 255.0
#     X_test = X_test / 255.0

#     y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
#     y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

#     train_datagen = ImageDataGenerator(
#         rotation_range=20,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         horizontal_flip=True,
#         zoom_range=0.2,
#         shear_range=0.2,
#         fill_mode='nearest'
#     )

#     train_generator = train_datagen.flow(
#         X_train, y_train, batch_size=BATCH_SIZE, shuffle=True
#     )

#     model = create_improved_model(num_classes)
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#         loss="categorical_crossentropy",
#         metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
#     )

#     early_stopping = EarlyStopping(
#         monitor="val_loss", patience=15, restore_best_weights=True
#     )
#     lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=7)

#     history = model.fit(
#         train_generator,
#         steps_per_epoch=len(X_train) // BATCH_SIZE,
#         epochs=EPOCHS,
#         validation_data=(X_test, y_test),
#         callbacks=[early_stopping, lr_scheduler]
#     )

#     test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test, verbose=2)
#     print(f"\nTest accuracy: {test_acc}")
#     print(f"Test precision: {test_precision}")
#     print(f"Test recall: {test_recall}")

#     os.makedirs(MODEL_DIR, exist_ok=True)
#     model_path = os.path.join(MODEL_DIR, "improved_balanced_asl_model.h5")
#     model.save(model_path)
#     print(f"Model saved successfully: {model_path}")

#     os.makedirs(LABELS_DIR, exist_ok=True)
#     label_encoder_path = os.path.join(LABELS_DIR, "label_encoder.pkl")
#     with open(label_encoder_path, "wb") as f:
#         pickle.dump(le, f)
#     print(f"Label Encoder saved successfully: {label_encoder_path}")

# if __name__ == "__main__":
#     main()


# training script

In [ ]:
# import os
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.preprocessing.image import load_img, img_to_array
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
# from tensorflow.keras.models import Model, model_from_json
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import to_categorical
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from PIL import Image, UnidentifiedImageError
# import pickle

# # Configuration
# IMG_SIZE = 227
# BATCH_SIZE = 32
# EPOCHS = 100  # You can adjust the number of epochs
# DATA_DIR = r"/content/drive/My Drive/ASL_to_Text_Project/data"
# IMAGES_DIR = os.path.join(DATA_DIR, 'images') # Path to images folder
# MODEL_DIR = r"/content/drive/My Drive/ASL_to_Text_Project/models"
# LABELS_DIR = r"/content/drive/My Drive/ASL_to_Text_Project/data/labels"

# def create_model(num_classes, input_shape=(IMG_SIZE, IMG_SIZE, 3)):
#     base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

#     x = base_model.output
#     x = GlobalAveragePooling2D()(x)
#     x = Dense(1024, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     outputs = Dense(num_classes, activation='softmax')(x)

#     model = Model(inputs=base_model.input, outputs=outputs)

#     # Freeze the base model layers
#     for layer in base_model.layers:
#         layer.trainable = False

#     return model

# def load_data(data_dir, allowed_extensions=('.jpg', '.jpeg', '.png')):
#     images = []
#     labels = []
#     for label in os.listdir(data_dir):
#         if label.startswith('.'):  # Skip hidden files/directories
#             continue
#         label_dir = os.path.join(data_dir, label)
#         if os.path.isdir(label_dir):
#             for img_name in os.listdir(label_dir):
#                 if img_name.startswith('.'):  # Skip hidden files within label directory
#                     continue
#                 if not img_name.lower().endswith(allowed_extensions):
#                     print(f"Skipping unsupported file: {img_name}")
#                     continue
#                 try:
#                     img_path = os.path.join(label_dir, img_name)
#                     img = Image.open(img_path)
#                     img.verify()
#                     img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
#                     img_array = img_to_array(img)
#                     images.append(img_array)
#                     labels.append(label)
#                 except (IOError, UnidentifiedImageError) as e:
#                     print(f"Error loading image {img_name}: {e}")
#                     continue
#     return np.array(images), np.array(labels)

# # --- Main Execution ---

# # Mount Google Drive (if using Colab)
# from google.colab import drive
# drive.mount('/content/drive')

# # Load and preprocess data
# X, y = load_data(IMAGES_DIR) # Load from the correct directory
# print(f"Number of images loaded: {len(X)}")
# X = X / 255.0  # Normalize pixel values

# # Encode labels
# le = LabelEncoder()
# y_encoded = le.fit_transform(y)
# num_classes = len(le.classes_)

# # Split data
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y_encoded, test_size=0.2, random_state=42
# )

# # Convert to categorical
# y_train = to_categorical(y_train, num_classes)
# y_test = to_categorical(y_test, num_classes)

# # Create and compile model
# model = create_model(num_classes)
# model.compile(
#     optimizer=Adam(learning_rate=0.0001),
#     loss="categorical_crossentropy",
#     metrics=["accuracy"],
# )

# # Data augmentation
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
#     zoom_range=0.2,
#     shear_range=0.2,
#     fill_mode="nearest",
# )

# # Train the model
# history = model.fit(
#     datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
#     steps_per_epoch=len(X_train) // BATCH_SIZE,
#     epochs=EPOCHS,
#     validation_data=(X_test, y_test),
#     callbacks=[
#         tf.keras.callbacks.ModelCheckpoint(
#             filepath=os.path.join(
#                 MODEL_DIR, "asl_model_{epoch:02d}_{val_accuracy:.2f}.keras"
#             ),
#             monitor="val_accuracy",
#             save_best_only=True,
#             mode="max",
#         )
#     ],
# )

# # Save the entire model
# model.save(os.path.join(MODEL_DIR, "asl_model.h5"))

# # Save the label encoder
# with open(os.path.join(LABELS_DIR, 'label_encoder.pkl'), 'wb') as f:
#   pickle.dump(le, f)